# Opening Chlorophyll-a NC File from NASA Ocean Color L2 repo


Step 1: Go to https://oceancolor.gsfc.nasa.gov/cgi/browse.pl?sen=amod

Step 2: Select a region from top right scroll menu and click "Find swaths"

Step 3: Click "Order Data" button on the new page that loads after Step 2

Step 4: Login with creds if you haven't already

In [31]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import netCDF4 as nc
import os

In [3]:
from matplotlib import pyplot as plt
%matplotlib inline
# set the font globally
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2
# update the overall font size
plt.rcParams.update({'font.size':11})
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


In [4]:
# txt = 'AQUA_MODIS.20020712.L3m.DAY.CHL.x_chlor_a.nc'
# print(txt.split('.')[1])

## Generation of RGB Images from NC files

In [5]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

directory = 'algae_dataset'

for filename in os.listdir(directory):
    fn = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(fn):
      ds = nc.Dataset(fn)
      fig, ax = plt.subplots(1,1,dpi = 159, figsize = (4,4))
      # divider = make_axes_locatable(ax)
      # cax = divider.append_axes('right', size='5%', pad=0.05)
      im = ax.imshow(ds.variables['chlor_a'][:].data)
      ax.invert_xaxis()
      ax.invert_yaxis()
      # fig.colorbar(im, cax=cax)
      isExist = os.path.exists('RGB_figures/')
      if not isExist:
        os.makedirs('RGB_figures/')
      fig.savefig('RGB_figures/'+filename.split('.')[1])
      
      plt.close(fig)
      

## Generation of Grayscale Images from RGB Images

In [3]:
from PIL import Image

directory = 'RGB_figures'

for filename in os.listdir(directory):
    fn = os.path.join(directory, filename)
    img = Image.open(fn).convert('L')
    isExist = os.path.exists('Grayscale_figures/')
    if not isExist:
        os.makedirs('Grayscale_figures/')
    img.save('Grayscale_figures/'+filename)

## Image Object

In [6]:
class ImageObj:
    def __init__(self, file_path):
        self.file_path = file_path
    
    def get_date(self):
        date = os.path.basename(self.file_path).split('.')[0]
        return date[:4]+'/'+date[4:6] + '/'+date[6:]
    
    def get_pixels(self):
        image = Image.open(self.file_path)
        return image.load()

## Split the dataset for Training, Validation and Testing

In [9]:
train = []
test = []
validation = []
for filename in os.listdir('Grayscale_figures'):
    file_path = os.path.join(directory, filename)
    img_obj = ImageObj(file_path)
    if len(train) <= len(os.listdir('Grayscale_figures'))*0.7:
        train.append(img_obj)
    elif len(validation) <= len(os.listdir('Grayscale_figures'))*0.15:
        validation.append(file_path)
    else:
        test.append(file_path)

In [11]:
print(len(train))
print(len(validation))
print(len(test))

5136
1101
1100
